In [ ]:
#####################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
##########################################################################################
# Using SD models (MLP, LSTM) predict on the whole subject's video (not just the test set partition)
##########################################################################################

In [1]:
###############################################################################################################
# MLP
###############################################################################################################

import numpy as np
import time
import glob
import matplotlib.pyplot as plt
from keras.models import load_model
from postprocessingutils import save_predictions_no_eval

# Load testing VIDs (4)
TE_folder = 'TrainingExamples_16kHz'
ds = np.load('./../Dataset/'+TE_folder+'/Dataset_split/split_masks_all.npz')
test_VIDs  = ds['test_VIDs']
test_VIDs_ind_cnts = ds['test_VIDs_ind_cnts']
print "Predicting whole video for: ", test_VIDs
print np.sum(test_VIDs_ind_cnts)

tuning_type = '1_35_AF26'
AF_type = 'AF_logFB26_norm'
model_checkpoint_path_prefix = './ModelCheckpoints/MLP_SD/'
save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/MLP_SD/'

PF = np.load('./../Dataset/'+TE_folder+'/te_PF_smooth_LPBF_4.0.npz')['PF_smooth_LPBF']
PF = PF[:, :11]

AF = np.load('./../Dataset/'+TE_folder+'/te_'+AF_type+'.npz')[AF_type]
N_features = AF.shape[1]

SEGMENT_LEN = 300

#######################
# Load the dataset split  
ds = np.load('./../Dataset/'+TE_folder+'/Dataset_split/split_masks_all.npz')
test_mask  = ds['test_mask']

X_test  = AF[test_mask]
del AF
Y_test  = PF[test_mask]
del PF

print X_test.shape

offset = 0
for VID, N_VID_samples in zip(test_VIDs, test_VIDs_ind_cnts):
    
    SID = VID[:5]
    #print SID
    
    #######################
    # Load best model for this SID
    model_name = sorted(glob.glob( model_checkpoint_path_prefix + tuning_type + '/m_' + SID + '_*' ))[-1] # take best model = last checkpointed
    test_model_name = model_name
    print "Loading BEST model from:", test_model_name
    model = load_model(test_model_name)
    
    ###############################################################################################################
    # Predict on testing set
    test_batch_size = len(X_test[offset:offset+N_VID_samples])
    if offset == 0:
        Y_test_pred = model.predict(X_test[offset:offset+N_VID_samples], batch_size=test_batch_size, verbose=1)
    else:
        pred = model.predict(X_test[offset:offset+N_VID_samples], batch_size=test_batch_size, verbose=1)
        Y_test_pred = np.concatenate( (Y_test_pred, pred), axis=0)
        
    offset += N_VID_samples

save_predictions_no_eval(save_results_path_prefix + 'WHOLEtest_MLP_SD', 
                         X_test, Y_test, Y_test_pred, 'MLP_SD', SEGMENT_LEN, test_VIDs, test_VIDs_ind_cnts, 
                        )


/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Predicting whole video for:  ['PID18Task2' 'PID18Task3' 'PID23Task2' 'PID23Task3']
21390
(21390, 26)
Loading BEST model from: ./ModelCheckpoints/MLP_SD/1_35_AF26/m_PID18_1000_0.0054_0.0051.hdf5
4687/4687 [==============================] - 0s 37us/step
Loading BEST model from: ./ModelCheckpoints/MLP_SD/1_35_AF26/m_PID18_1000_0.0054_0.0051.hdf5
5871/5871 [==============================] - 0s 9us/step
Loading BEST model from: ./ModelCheckpoints/MLP_SD/1_35_AF26/m_PID23_1000_0.0015_0.0014.hdf5
5801/5801 [==============================] - 0s 13us/step
Loading BEST model from: ./ModelCheckpoints/MLP_SD/1_35_AF26/m_PID23_1000_0.0015_0.0014.hdf5
5031/5031 [==============================] - 0s 19us/step
Saved (raw & smoothed predictions WITHOUT results) to: ./../Dataset/TrainingExamples_16kHz/Results/MLP_SD/WHOLEtest_MLP_SD


In [2]:
###############################################################################################################
# LSTM
###############################################################################################################

import numpy as np
import time
import glob
import matplotlib.pyplot as plt
from keras.models import load_model
from postprocessingutils import save_predictions_no_eval

AF_type = 'AF_logFB26_norm'
SEG_folder = 'Segments_logFB26'
TE_folder = 'TrainingExamples_16kHz'
# Load segmented data, already split
X_test_RT = np.load('./../Dataset/'+TE_folder+'/' + SEG_folder + '/seg_test_RT.npz')['X'] 
Y_test_RT = np.load('./../Dataset/'+TE_folder+'/' + SEG_folder + '/seg_test_RT.npz')['Y'] 
test_VIDs =  np.load('./../Dataset/'+TE_folder+'/' + SEG_folder + '/seg_test.npz')['test_VIDs']
print "Test VIDs:", test_VIDs, "\n"
N_test_RT_seg =  np.load('./../Dataset/'+TE_folder+'/' + SEG_folder + '/seg_test_RT.npz')['N_test_RT_seg']
print "N_test_RT_seg:", N_test_RT_seg, "\n"

print X_test_RT.shape

N_runs = 1
SEGMENT_LEN = 300 # == X_train.shape[1]
N_features = 26
N_targets  = 11
N_LSTM_units = 12 # BEST FOUND in LSTM_SI
save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/LSTM_SD/'
model_checkpoint_path_prefix = './ModelCheckpoints/LSTM_SD/{:d}_{:d}_{:d}_'.format(N_runs, N_features, N_LSTM_units)


offset = 0
for VID, N_VID_samples in zip(test_VIDs, N_test_RT_seg):
    
    SID = VID[:5]
    #print SID
    
    #######################
    # Load best model for this SID
    test_model_name = sorted(glob.glob(model_checkpoint_path_prefix + SID + '/m_*'))[-1]
    print "Loading BEST model from:", test_model_name
    model = load_model(test_model_name)
    
    ###############################################################################################################
    # Predict on testing set
    test_batch_size = len(X_test_RT[offset:offset+N_VID_samples])
    
    if offset == 0:
        pred = model.predict(X_test_RT[offset:offset+N_VID_samples], batch_size=test_batch_size, verbose=1)
        Y_test_pred = pred[:, -1, :] # last item from each segment is the (ONLINE) final prediction
        Y_test_true = Y_test_RT[offset:offset+N_VID_samples, -1, :]
        X_test_RT_last = X_test_RT[offset:offset+N_VID_samples, -1, :]
    else:
        pred = model.predict(X_test_RT[offset:offset+N_VID_samples], batch_size=test_batch_size, verbose=1)
        pred = pred[:, -1, :]
        Y_test_pred = np.concatenate( (Y_test_pred, pred), axis=0)
        Y_test_true = np.concatenate( (Y_test_true, Y_test_RT[offset:offset+N_VID_samples, -1, :]), axis=0)
        X_test_RT_last = np.concatenate( (X_test_RT_last, X_test_RT[offset:offset+N_VID_samples, -1, :]), axis=0)
        
    offset += N_VID_samples

save_predictions_no_eval(save_results_path_prefix + 'WHOLEtest_LSTM_SD', 
                         X_test_RT_last, Y_test_true, Y_test_pred, 'LSTM_SD', SEGMENT_LEN, test_VIDs, N_test_RT_seg, 
                        )
print X_test_RT_last.shape, Y_test_true.shape, Y_test_pred.shape

Test VIDs: ['PID18Task2' 'PID18Task3' 'PID23Task2' 'PID23Task3'] 

N_test_RT_seg: [4687 5871 5801 5031] 

(21390, 300, 26)
Loading BEST model from: ./ModelCheckpoints/LSTM_SD/1_26_12_PID18/m_0500_0.0054_0.0047.hdf5
4687/4687 [==============================] - 1s 152us/step
Loading BEST model from: ./ModelCheckpoints/LSTM_SD/1_26_12_PID18/m_0500_0.0054_0.0047.hdf5
5871/5871 [==============================] - 1s 148us/step
Loading BEST model from: ./ModelCheckpoints/LSTM_SD/1_26_12_PID23/m_0500_0.0020_0.0020.hdf5
5801/5801 [==============================] - 1s 145us/step
Loading BEST model from: ./ModelCheckpoints/LSTM_SD/1_26_12_PID23/m_0500_0.0020_0.0020.hdf5
5031/5031 [==============================] - 1s 164us/step
Saved (raw & smoothed predictions WITHOUT results) to: ./../Dataset/TrainingExamples_16kHz/Results/LSTM_SD/WHOLEtest_LSTM_SD
(21390, 26) (21390, 11) (21390, 11)
